## Animation button

In [ ]:
# Imports
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output, callback
import numpy as np
import plotly.offline as pyo
import plotly.graph_objs as go
import dash_bootstrap_components as dbc
import dash_daq as daq

# Helper function to normalize to an array to a range (x,y)
def normalize_range(array, x, y):
    m = min(array)
    range = max(array) - m
    array = (array - m) / range
    range2 = y - x
    normalized = (array*range2) + x
    return list(normalized)

# Datasets and variables
snapshot = pd.read_csv("new_data.csv")
vehicle_types = sorted(snapshot["vehicle_type"].unique())
timeline = snapshot["timestep"].unique()
snapshot["vehicle_type"] = pd.Categorical(snapshot["vehicle_type"], categories=["electric","fuel"], ordered=True)
snapshot = snapshot.infer_objects()
snapshot["timestep"] = pd.to_datetime(snapshot["timestep"], format="%M").dt.minute
#snapshot["timestep"] = pd.to_timedelta(snapshot["timestep"], freq="min")
# Center point for heatmap
center_point= dict(lon=snapshot["lon"].mean(), lat = snapshot["lat"].mean())
# Style
external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.BOOTSTRAP]
# Hover layout
hovers=dict(bgcolor="white", font_size=16)

# Emission types
emission_types = ["Noise","Carbon monoxide","Carbon dioxide", "Hydrocarbon", "Nitrogen oxides", "Particle matters"]
emission_cols = {"Noise": "vehicle_noise", "Carbon monoxide": "vehicle_CO","Carbon dioxide": "vehicle_CO2", "Hydrocarbon": "vehicle_HC", "Nitrogen oxides": "vehicle_NOx", "Particle matters": "vehicle_PMx"}
emission_units = {"Noise": "dB", "Carbon monoxide": "mg","Carbon dioxide": "mg", "Hydrocarbon": "mg", "Nitrogen oxides": "mg", "Particle matters": "mg"}
emission_maxes = 0.7*snapshot.max(axis=0, numeric_only=True)
emission_maxes = emission_maxes.to_dict()

# Main component
anim_app = Dash(__name__, external_stylesheets=external_stylesheets)
anim_app.layout = html.Div([

        # Toggles and texts for parameters
        html.Div([

            # Title
            html.H2("Visualization parameters"),
            # Emission dropdown
            html.H3("Emission type", style={"marginTop": "20px", "display": "inline-block"}),
            dbc.Button(
                [html.I(className="bi bi-info-circle-fill me-2")],
                id="emission-info",
                color="white",
                title="""SUMO can simulate varying emissions with the HBEFA v4.2.-based emissions model.
Choose which one you wish to analyze in the visualisation.""",
                n_clicks=0,
                style={"marginLeft": "20px", "marginBottom": "10px", "display": "inline-block"},
            ),
            dcc.Dropdown(
                emission_types,
                "Carbon dioxide",
                id="crossfilter-emission",
                style={"width": "80%", "font-size": 18, "marginTop": "5px", "display": "inline-block"}),
            # Timeline type
            html.H3("Timeline type", style={"marginTop": "20px", "display": "inline-block"}),
                dbc.Button(
                [html.I(className="bi bi-info-circle-fill me-2")],
                id="timeline-info",
                color="white",
                title="""This setting controls how the results are aggregated over the timesteps.
Sum sums the simulation results over each timestep.
Average averages the simulation results over each timestep.""",
                n_clicks=0,
                style={"marginLeft": "20px", "marginBottom": "10px", "display": "inline-block"},
                ),
                dcc.RadioItems(
                ["Sum", "Average"],
                "Sum",
                id="crossfilter-timeline-type",
                labelStyle={"font-size": 18, "marginTop": "10px", "marginRight": "15px", "display": "inline-block"},
                inputStyle={"margin-right": "5px"}
                ),
            # Timestep interval
            html.H3("Timestep interval", style={"marginTop": "20px", "display": "inline-block"}),
            dbc.Button(
                [html.I(className="bi bi-info-circle-fill me-2")],
                id="timestep-info",
                color="white",
                title="""This setting controls what is the aggregation interval for the time and consequently the number of temporal data points.
By default, the results are aggregated with an interval of 1 minute, resulting in 60 data points for 1 hour of data.
For example, choosing 30 minutes as the aggregation interval instead would result in two data points for 1 hour of data.""",
                n_clicks=0,
                style={"marginLeft": "20px", "marginBottom": "10px"},
            ),
            html.Div([
            daq.NumericInput(
            value=1,
            id="crossfilter-timestep-range",
            min=timeline.min(),
            max=timeline.max(),
            #type="number",
            #placeholder="1",
            style={"width": "10%", "display": "inline-block"}),
            html.P("minute(s)", style={"marginLeft": "20px", "font-size": 18, "display": "inline-block"})
            ]),
        ], style={"float": "left", "width": "35%", "marginTop": "20px", "marginLeft": "30px", "marginBottom": "30px"}),

        # Dividing line between the params and plots
        html.Hr(style={"width": "99%"}),

        # Title for results
        html.H2("Traffic simulation results", style={"marginLeft": "30px", "marginBottom": "20px"}),

        # Left-side plots
        html.Div([
            # Title
            html.H3("Network emissions", style={"marginLeft": "50px", "display": "inline-block"}),
            # Info button for the heatmap
            dbc.Button(
                [html.I(className="bi bi-info-circle-fill me-2")],
                id="heatmap-info",
                color="white",
                title="""The heatmap
Shows how the emissions and amount of vehicles 
are distributed in different roads in the network.
By clicking on a point in the heatmap, 
you can display road-specific results on the right.

Encoding:
Radius: the amount of vehicles
Colour: the generated emissions""",
                n_clicks=0,
                style={"marginLeft": "30px", "marginBottom": "10px", "display": "inline-block"},
            ),
            # The heatmap
            dcc.Loading(dcc.Graph(id="x-heatmap"), type="cube", style={"marginBottom": "30px", "marginTop": "150px"}),
        ], style={"float": "left", "width": "45%", "marginLeft": "30px"}),

        
        # Right-side plots    
        html.Div([
            
            # Infos
            html.Div([
            # Title
            html.H3("Vehicle type emissions", style={"marginTop": "10px", "marginBottom": "30px", "display": "inline-block"}),
            # Info button for histogram
            dbc.Button(
                [html.I(className="bi bi-info-circle-fill me-2")],
                id="barplot-info",
                color="white",
                title="""The upper bar plot 
Shows the emission shares among different vehicle types
in the network (or a road, when clicking a point on the heatmap).
Encoding:
X-axis: the magnitude of emissions
Y-axis: the vehicle type
Length: the emissions generated by all vehicles of one type
Colour: the average emissions per vehicle for each vehicle type

The lower stream graph 
Shows how the emissions are generated temporally
in the network (or a road, when clicking a point on the heatmap).
Encoding:
X-axis: the timesteps
Y-axis: the magnitude of emissions
Colour: different vehicle types
Area: the emissions generated by all vehicles of one type""",
                n_clicks=0,
                style={"marginLeft": "20px", "marginBottom": "10px"}
            ),
            ], style={"marginTop": "-10px"}),
            # Reset button
            html.Button(
            "Reset location",
            id="reset_button",
            n_clicks=0,
            style={"marginBottom": "10px", "marginTop": "20px", "marginLeft": "80px"}),
            # Histogram
            dcc.Graph(id="y-road", style={"marginBottom": "10px"}),
            # Stream graph
            dcc.Graph(id="y-flow"),
        ], style={"float": "right", "width": "45%", "marginLeft": "30px"}),

# Main component
])

# Callbacks
@anim_app.callback(
    Output("x-heatmap", "figure"), 
    Input("crossfilter-emission", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"))
def get_heatmap(emission_name, timestep_range, timeline_type):
    zoom_level = 13
    if emission_name is None:
        emission_name = "Carbon dioxide"
    emission = emission_cols[emission_name]
    func = "sum"
    df = snapshot
    if timestep_range != 1:
        bins = np.arange(timeline.min(), timeline.max(), timestep_range, dtype=int)
        df['timestep'] = pd.cut(df['timestep'], bins=bins, labels=bins[:-1], right=False, include_lowest=True)
        if timeline_type == "Average":
            func = "mean"
        df = df.groupby(["vehicle_lane", "timestep"], observed=False).agg({emission: func, "amount": func, "lon": "first", "lat": "first"}).reset_index()
    fig = px.density_map(df, lat = "lat", lon = "lon", z = emission,
                            radius = normalize_range(df["amount"], 10, 18),
                            center = center_point,
                            zoom = zoom_level,
                            map_style = "open-street-map",
                            range_color = (0,emission_maxes[emission]),
                            hover_data = {"vehicle_lane": False, "amount": True, "lon": False, "lat": False, emission: True},
                            animation_frame="timestep",
                            labels={emission: emission_name, "vehicle_type": "Vehicle type", "amount": "Vehicle amount", "timestep": "Timestep"},
                            width=600,
                            height=600
                            )
    sliders = [dict(
    active=0,
    currentvalue={"prefix": "Time: "},
    font={"size": 18}
    )]
    fig.update_traces(unselected_marker_opacity=0.3, selector=dict(type="scatter"))
    fig.update_layout(hovermode="closest", sliders=sliders, title_x=0.12, hoverlabel=hovers)
    fig.layout["coloraxis"]["colorbar"]["title"] = f"{emission_name} ({emission_units[emission_name]})"
    fig["layout"]["uirevision"] = "Default"
    return fig

@anim_app.callback(
    Output("y-road", "figure"),
    Output("y-flow", "figure"),
    Input("x-heatmap", "clickData"),
    Input("crossfilter-emission", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"))
def get_barplot(clickData, emission_name, timestep_range, timeline_type):
    road = 0
    road_hist = 0
    road_name = "Network"
    if emission_name is None:
        emission_name = "Carbon dioxide"
    emission = emission_cols[emission_name]
    df = snapshot
    func = "mean"
    if timestep_range != 1:
        bins = np.arange(timeline.min(), timeline.max(), timestep_range, dtype=int)
        df['timestep'] = pd.cut(df['timestep'], bins=bins, labels=bins[:-1], right=False, include_lowest=True)
        if timeline_type == "Average":
            func = "mean"
    df = df.groupby(["vehicle_type", "vehicle_lane", "timestep"], observed=False).agg({emission: func, "amount": func}).reset_index()
    if clickData is None:
        road = df
    else:
        road_id = clickData["points"][0]["customdata"][0]
        road_name = "Road"
        road = df[df["vehicle_lane"] == road_id]
    road_hist = road.groupby(["vehicle_type"], observed=False).agg({emission: "sum", "amount": "sum"}).reset_index()
    road_stream = road.groupby(["vehicle_type", "timestep"], observed=False).agg({emission: "sum", "amount": "sum"}).reset_index()
    road_stream.sort_values(by=["vehicle_type"])
    road_hist.sort_values(by=["vehicle_type"])
    road_hist_non_zero = road_hist[road_hist[emission] != 0]
    road_hist["average_vehicle"] = np.round(road_hist_non_zero[emission]/road_hist_non_zero["amount"], 4)
    average_dict = {road_hist["vehicle_type"][0]: road_hist["average_vehicle"][0], road_hist["vehicle_type"][1]: road_hist["average_vehicle"][1]}
    road_stream["average_vehicle"] = road_stream["vehicle_type"].map(average_dict)
    title = "<b>Location: {}</b>".format(road_name)
    hist = px.bar(road_hist, x=emission, y="vehicle_type", color="average_vehicle", hover_data="amount", title=title, labels={"average_vehicle": "Average vehicle emission", emission: emission_name, "vehicle_type": "Vehicle type", "amount": "Vehicle amount"})
    hist.update_layout(hoverlabel=hovers, title_x=0.11, bargap=0.5, xaxis_title=f"{emission_name} ({emission_units[emission_name]})", yaxis_title="Vehicle type", xaxis=dict(range=[0, snapshot[emission].max()]))
    stream = px.area(road_stream, x="timestep", y=emission, color="vehicle_type", color_discrete_sequence=px.colors.sequential.Plasma_r, hover_data=["average_vehicle", "amount"],  labels={"average_vehicle": "Average vehicle emission", emission: emission_name, "vehicle_type": "Vehicle type", "amount": "Amount"})
    stream.update_layout(hoverlabel=hovers, title_x=0.11, yaxis_title=f"{emission_name} ({emission_units[emission_name]})")
    return hist, stream

@anim_app.callback(Output("x-heatmap","clickData"),
             [Input("reset_button","n_clicks")])
def update(reset):
    return None

anim_app.run(debug=True)